# AWS Example - Workspace

This notebook demonstrates using AWS CLI inside a CodingBooth Workspace container.

## How Credentials Work

Your AWS credentials are mounted from your host machine using the **home seeding** pattern:

1. Host `~/.aws/` is mounted read-only to `/tmp/cb-home-seed/.aws/`
2. At container startup, files are copied to `/home/coder/.aws/`
3. AWS CLI uses credentials from `/home/coder/.aws/`

This means:
- Your host credentials stay protected (read-only mount)
- The container has a writable copy
- Any changes inside the container don't affect your host

## 1. Check AWS CLI Version

In [ ]:
aws --version

## 2. Verify Credentials

Run this command to check if your credentials are configured correctly.
If it returns your account info, you're all set!

In [ ]:
aws sts get-caller-identity

### If Credentials Don't Work

If the above command fails, you need to configure AWS credentials.

**📚 Helpful Resources:**
- [AWS CLI Configuration Basics](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-files.html) - Official documentation
- [Setting Up AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-quickstart.html) - Quick start guide
- [Creating IAM Access Keys](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_credentials_access-keys.html) - How to get your keys
- [AWS CLI Setup Tutorial (YouTube)](https://www.youtube.com/watch?v=Rp-A84oh4G8) - Step-by-step video guide

**Option 1: Configure on your HOST machine (recommended)**

Run this on your host (not in the container), then restart the container:
```bash
aws configure
```

**Option 2: Configure inside the container (temporary)**

Uncomment and run the cell below (changes won't persist to host):

In [ ]:
# Uncomment and set your credentials (temporary, container-only)
# export AWS_ACCESS_KEY_ID="your-access-key"
# export AWS_SECRET_ACCESS_KEY="your-secret-key"
# export AWS_DEFAULT_REGION="us-east-1"

## 3. List S3 Buckets

Once credentials are configured, try listing your S3 buckets:

In [ ]:
aws s3 ls

## 4. List Objects in a Bucket

Replace `your-bucket-name` with an actual bucket name from the list above:

In [ ]:
# aws s3 ls s3://your-bucket-name/

## 5. Other AWS Commands

Here are some other useful AWS commands to try:

In [ ]:
# List EC2 instances
# aws ec2 describe-instances --query 'Reservations[*].Instances[*].[InstanceId,State.Name,InstanceType]' --output table

In [ ]:
# List Lambda functions
# aws lambda list-functions --query 'Functions[*].[FunctionName,Runtime]' --output table

In [ ]:
# List IAM users
# aws iam list-users --query 'Users[*].[UserName,CreateDate]' --output table

## 6. Check Current Configuration

In [ ]:
# Show current AWS configuration
aws configure list

In [ ]:
# Show configured profiles
cat ~/.aws/config 2>/dev/null || echo "No config file found"

---

## Summary

| Feature     | Description                                       |
| ----------- | ------------------------------------------------- |
| Credentials | Mounted from host `~/.aws/`                       |
| Pattern     | Home seeding (read-only mount, copied at startup) |
| Security    | Host files protected, container can write freely  |

For more information, see the [WorkSpace README](https://github.com/NawaMan/WorkSpace).

# What is OK (and NOT OK) to Commit in AWS Tutorials

## ✅ OK to commit (safe)

| Item                             | Example                                        | Why it’s OK                                  |
|----------------------------------|------------------------------------------------|----------------------------------------------|
| AWS CLI config (no secrets)      | `~/.aws/config` or `.example`                  | Regions, output format, role references only |
| Profile names                    | `tutorial`, `demo`                             | Not sensitive                                |
| AWS account ID                   | `123456789012`                                 | Identifier only, not a secret                |
| IAM role ARN                     | `arn:aws:iam::123456789012:role/tutorial-role` | Cannot be used to authenticate               |
| IAM user ARN                     | `arn:aws:iam::123456789012:user/tutorial-user` | Identity only                                |
| IAM policy JSON                  | `policy.json`                                  | Meant to be shared                           |
| CloudFormation / Terraform / CDK | `template.yaml`, `main.tf`                     | Infra definitions, not credentials           |
| AWS regions                      | `us-east-1`                                    | Public information                           |
| CLI commands & scripts           | `aws sts get-caller-identity`                  | No secrets involved                          |
| Masked outputs                   | `AIDA****`                                     | Safe for examples                            |
| README instructions              | Setup steps                                    | Descriptive only                             |


---

## ⚠️ OK with care (sanitize for public repos)

| Item                        | Example                     | Why to be careful          |
|-----------------------------|-----------------------------|----------------------------|
| IAM user names              | `alice-admin`               | May reveal real names      |
| Role names                  | `billing-prod-admin`        | Exposes internal structure |
| Resource names              | `acme-payments-prod-bucket` | Business context leakage   |
| Unmasked STS output         | Full `get-caller-identity`  | Low risk but unnecessary   |
| Account IDs in public repos | Real account number         | Account fingerprinting     |
| Architecture comments       | “Prod account does X”       | Helps social engineering   |

---

## ❌ NOT OK to commit (never)

| Item                     | Example                     | Why it’s dangerous             |
|--------------------------|-----------------------------|--------------------------------|
| Access keys              | `AKIA...`                   | Full authentication            |
| Secret access keys       | `wJalrXUtnFEMI...`          | Full authentication            |
| Session tokens           | `IQoJb3JpZ2lu...`           | Temporary but powerful         |
| `.aws/credentials`       | Credentials file            | Almost always contains secrets |
| `.env` with AWS vars     | `AWS_SECRET_ACCESS_KEY=...` | Easy to leak                   |
| Hard-coded creds in code | SDK config                  | Instantly exploitable          |
| Screenshots with secrets | Terminal output             | Still leakable                 |
| Logs containing creds    | Debug output                | Often overlooked               |


---

## Rule of thumb

- If it can authenticate → **do not commit**
- If it only identifies resources/accounts → **usually OK**
- If you’d regret it being public forever → **sanitize it**

---

## Recommended repo pattern (IAM user tutorials)

| File                  | Commit?           |
|-----------------------|-------------------|
| `.aws/config.example` | ✅ Yes            |
| `.aws/credentials`    | ❌ No (gitignore) |
| `policy.json`         | ✅ Yes            |
| `README.md`           | ✅ Yes            |
| `.env.example`        | ✅ Yes            |
| `.env`                | ❌ No (gitignore) |



# `.gitignore` for AWS project

```
# AWS credentials
.aws/credentials
.aws/credentials.*
.aws/config.local

# Environment variables
.env
.env.*
!.env.example

# Terraform
.terraform/
*.tfstate
*.tfstate.*

# Logs & debug
*.log

# OS / editor
.DS_Store
.idea/
.vscode/

```


#  Prevent Accidental AWS Sensitive Information Commit

Use one of the following...

### Git Hook
**`.git/hooks/pre-commit`**

```shell
#!/usr/bin/env bash
set -e

echo "🔍 Running AWS secret scan..."

# Block AWS access keys
if git diff --cached | grep -E 'AKIA[0-9A-Z]{16}' > /dev/null; then
  echo "❌ ERROR: AWS Access Key detected."
  echo "Remove it before committing."
  exit 1
fi

# Block AWS secret access keys
if git diff --cached | grep -E 'AWS_SECRET_ACCESS_KEY' > /dev/null; then
  echo "❌ ERROR: AWS Secret Access Key detected."
  exit 1
fi

# Block credentials file
if git diff --cached --name-only | grep -E '^\.aws/credentials' > /dev/null; then
  echo "❌ ERROR: .aws/credentials must not be committed."
  exit 1
fi

echo "✅ No AWS secrets detected."

```

Then :

```shell
chmod +x .git/hooks/pre-commit
git config core.hooksPath .githooks
```

### Use `pre-commit` (need install)

**`.pre-commit-config.yaml`**

```yaml
repos:
  - repo: https://github.com/awslabs/git-secrets
    rev: v1.3.0
    hooks:
      - id: git-secrets
```

Then

```shell
pre-commit install
```